In [1]:
import os

In [2]:
%pwd

'c:\\Users\\jv3250328\\Downloads\\End_To_End_Machine_Learning_Project_MLOps\\research'

In [3]:
os.chdir("../")
print(os.getcwd())

c:\Users\jv3250328\Downloads\End_To_End_Machine_Learning_Project_MLOps


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from End_to_End_ML_Pipeline.constants import *
from End_to_End_ML_Pipeline.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [ ]:
import zipfile
from urllib import request as req
from End_to_End_ML_Pipeline import logger
from End_to_End_ML_Pipeline.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = req.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"File: {filename} downloaded with info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise

[2025-08-23 17:31:27,602: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-08-23 17:31:27,607: INFO: common]: yaml file: params.yaml loaded successfully
[2025-08-23 17:31:27,610: INFO: common]: Directory created at: artifacts
[2025-08-23 17:31:27,614: INFO: common]: Directory created at: artifacts/data_ingestion
[2025-08-23 17:31:29,791: INFO: 637199353]: File: artifacts/data_ingestion/data.zip downloaded with info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D715:2E7434:2213F7:2C7E12:68AA4141
Accept-Ranges: bytes
Date: Sat, 23 Aug 2025 22:31:30 GMT
Via: 1.1 varnish
X-Served-By: cache-bog-skbo2340073-BOG
X-Cac